In [23]:
import pandas as pd
import numpy as np

df=pd.read_csv("/home/Ima/work/Dataset/processed_burnout_dataset.csv")

df.head()

,EmployeeID,Age,YearsAtCompany,WorkHoursPerWeek,BurnoutLevel,JobSatisfaction,StressLevel,ProductivityScore,SleepHours,PhysicalActivityHrs,...,RemoteWork_Yes,HasMentalHealthSupport_No,HasMentalHealthSupport_Yes,HasTherapyAccess_No,HasTherapyAccess_Yes,SalaryRange_100K+,SalaryRange_40K-60K,SalaryRange_60K-80K,SalaryRange_80K-100K,SalaryRange_<40K
0,0.000000,0.756757,0.70,0.586207,0.263333,0.451111,0.941111,0.351111,0.60,0.79,...,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
1,0.000333,0.378378,0.05,1.000000,0.710000,0.111111,0.516667,0.304444,0.64,0.90,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,0.000667,0.189189,0.65,1.000000,0.677778,0.685556,0.522222,0.866667,0.24,0.97,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.001000,0.540541,0.75,0.034483,0.353333,0.306667,0.600000,0.410000,0.94,0.58,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4,0.001334,0.486486,0.30,0.137931,0.808889,0.148889,0.267778,0.124444,0.04,0.33,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [24]:
X = df.drop(['BurnoutRisk', 'EmployeeID'], axis=1)
y = df['BurnoutRisk']

In [25]:
from sklearn.model_selection import train_test_split

X = df.drop(columns=['BurnoutRisk'])
y = df['BurnoutRisk']   


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [26]:
#Decision Tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, roc_curve

dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train, y_train)
y_pred_dt = dt.predict(X_test)
y_prob_dt = dt.predict_proba(X_test)[:, 1]

In [27]:
#Train Random Forest
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
y_prob_rf = rf.predict_proba(X_test)[:, 1]

In [28]:
# k-NN
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
y_pred_knn = knn.predict(X_test)
y_prob_knn = knn.predict_proba(X_test)[:, 1]

In [30]:
def evaluate_model(name, y_true, y_pred, y_prob):
    print(f"--- {name} ---")
    print("Accuracy:", accuracy_score(y_true, y_pred))
    print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred))
    print("ROC AUC:", roc_auc_score(y_true, y_prob))
    print()

evaluate_model("Decision Tree", y_test, y_pred_dt, y_prob_dt)
evaluate_model("Random Forest", y_test, y_pred_rf, y_prob_rf)
evaluate_model("k-NN", y_test, y_pred_knn, y_prob_knn)


--- Decision Tree ---
Accuracy: 1.0
Confusion Matrix:
 [[399   0]
 [  0 201]]
ROC AUC: 1.0

--- Random Forest ---
Accuracy: 1.0
Confusion Matrix:
 [[399   0]
 [  0 201]]
ROC AUC: 1.0

--- k-NN ---
Accuracy: 0.6916666666666667
Confusion Matrix:
 [[351  48]
 [137  64]]
ROC AUC: 0.6523647427025275



In [31]:
#importand features from random forest
importances = rf.feature_importances_
feature_names = X.columns

feature_importance_df = pd.DataFrame({"Feature": feature_names, "Importance": importances}).sort_values(by="Importance", ascending=False)

top_3_features = feature_importance_df["Feature"].head(3).tolist()
print("\nTop 3 Selected Features:", top_3_features)



Top 3 Selected Features: ['BurnoutLevel', 'CareerGrowthScore', 'WorkLifeBalanceScore']


In [32]:
#dropping weak features
selected_features = ['BurnoutLevel', 'CareerGrowthScore', 'WorkLifeBalanceScore']
X_selected = df[selected_features]
y = df['BurnoutRisk']

from sklearn.model_selection import train_test_split
X_train_sel, X_test_sel, y_train_sel, y_test_sel = train_test_split(X_selected, y, test_size=0.2, random_state=42)

#Decision Tree
from sklearn.tree import DecisionTreeClassifier

dt_sel = DecisionTreeClassifier(random_state=42)
dt_sel.fit(X_train_sel, y_train_sel)
y_pred_dt_sel = dt_sel.predict(X_test_sel)
y_prob_dt_sel = dt_sel.predict_proba(X_test_sel)[:,1]

#Random forest
from sklearn.ensemble import RandomForestClassifier

rf_sel = RandomForestClassifier(n_estimators=100, random_state=42)
rf_sel.fit(X_train_sel, y_train_sel)
y_pred_rf_sel = rf_sel.predict(X_test_sel)
y_prob_rf_sel = rf_sel.predict_proba(X_test_sel)[:, 1]

#k-NN
from sklearn.neighbors import KNeighborsClassifier

knn_sel = KNeighborsClassifier(n_neighbors=5)
knn_sel.fit(X_train_sel, y_train_sel)
y_pred_knn_sel = knn_sel.predict(X_test_sel)
y_prob_knn_sel = knn_sel.predict_proba(X_test_sel)[:, 1]


evaluate_model("Decision Tree", y_test, y_pred_dt, y_prob_dt)
evaluate_model("Random Forest", y_test, y_pred_rf, y_prob_rf)
evaluate_model("k-NN", y_test, y_pred_knn, y_prob_knn)


--- Decision Tree ---
Accuracy: 1.0
Confusion Matrix:
 [[399   0]
 [  0 201]]
ROC AUC: 1.0

--- Random Forest ---
Accuracy: 1.0
Confusion Matrix:
 [[399   0]
 [  0 201]]
ROC AUC: 1.0

--- k-NN ---
Accuracy: 0.6916666666666667
Confusion Matrix:
 [[351  48]
 [137  64]]
ROC AUC: 0.6523647427025275



In [ ]:
#before dropping shows high acurracy which shows overfitting and after training with important features shows that the feautures were higly predictive